In [ ]:
#input must be options chain day1 - end

In [ ]:
!pip install python-dotenv
!pip install duckdb
!pip install tqdm
!pip install minio


In [25]:
import os,h5py
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from IPython.display import clear_output
import joblib
import random
import hashlib
from src.wgangp.utils import Scaler
from IPython.utils import io


In [2]:
# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

In [3]:
CSV_PATH     = './data/OptionsEOD.csv/'
#create @ part 1
PARQUET_PATH = './data/OptionsEOD.parquet'
#create @ part 2
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = [ 'UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
#create @ part 3
OTHER_COL = ['SYMBOL','QUOTE_UNIXTIME', 'EXPIRE_UNIX']

QUOTE_COL = ['C_BID',	'C_ASK',  'P_BID',	'P_ASK']
VEGA_COL =  ["C_VEGA","P_VEGA"] 
VOLUME_COL =  ["TOTAL_VOLUME","C_VOLUME","P_VOLUME"] 

SCALER_QUOTE_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in QUOTE_COL]
SCALER_VEGA_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VEGA_COL]
SCALER_VOLUME_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VOLUME_COL]
SCALER_OTHER_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ]


UNIQUE_KEYS = ['SYMBOL','EXPIRE_DATE']
H5_PATH = './data/OptTrainRenforcement/'
START = True#True#'2010-09'#True
BUCKET = 'tensorflow'

max_option_len = 16
#==== Create options_qoute
options_qoute = {}

In [4]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()  # By default, it loads the .env file from the current directory
import duckdb
con = duckdb.connect(database=':memory:')
con.execute(f"""
CREATE SECRET secretMinioProd (
    TYPE S3,
    KEY_ID '{os.getenv('MINIO_USER')}',
    SECRET '{os.getenv('MINIO_SECRET')}',
    ENDPOINT '{os.getenv('MINIO_HOST')}',
    USE_SSL 'false',
    URL_STYLE 'path'
);
""")

In [5]:
def strikeZero(df,v,qv,num_rm):   
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_arr = df[(df['QUOTE_DATE'] == qv['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]['STRIKE'].values
    
    if num_rm != 0 :
        df.loc[ (df['QUOTE_DATE'] == qv['QUOTE_DATE']) 
                & (df['SYMBOL'] == v['SYMBOL']) 
                & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
                & (  (df['STRIKE'].isin(filtered_arr[:num_rm])  )
                   | (df['STRIKE'].isin(filtered_arr[-num_rm:])   )
                  ) 
            , SCALER_COL    
        ] = 0

In [6]:
#============ example ===================

In [7]:
# EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
# EOD_CSV.head()

In [8]:
#==================================Create OptionsEOD.parquet part 1========================================

In [24]:

    for d_i,d in enumerate( os.listdir(CSV_PATH) ):
        print(f"[{d_i}/{len(os.listdir(CSV_PATH))}] : {d}                           " ,end='\r')
        print()
        ### tqdm_notebook
        for f_i,f in enumerate( tqdm_notebook(os.listdir(CSV_PATH+f"{d}/"), desc = 'processing',position = 0  ) ):
        #for f_i,f in enumerate( os.listdir(CSV_PATH+f"{d}/")[:20] ):
            
            DATA = pd.DataFrame([],columns=OTHER_COL+SCALER_COL)
            if f.endswith(".txt") :#or f.endswith(".csv"):
                ## load
                #print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f)
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )

                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)
                EOD_CSV = EOD_CSV.dropna(subset=SCALER_COL)
            
                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(str)
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #==== for each EXPIRE_DATE , SYMBOL
                keys_item = EOD_CSV[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
                for j,v in tqdm_notebook(keys_item.iterrows(), desc = 'pandas_item',position = 1  ):
                    
                    debug = {}
                    debug['symbo'] = v['SYMBOL']
                    debug['exdate'] = v['EXPIRE_DATE']
                
                    df_Item=EOD_CSV[(EOD_CSV['SYMBOL'] == v['SYMBOL']) & (EOD_CSV['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
                    keys_quote = df_Item[['QUOTE_DATE']].sort_values(by=['QUOTE_DATE']).drop_duplicates()[:]
                #====/===== for each QUOTE_DATE
                    for k,q_v in keys_quote.iterrows():
                        #print(f"""[{f_i}/{len(os.listdir(CSV_PATH+f"{d}/"))}] :{v['SYMBOL']} | {v['EXPIRE_DATE']} | {k}                      """ ,end='\r')
                        df_filter =df_Item[(df_Item['QUOTE_DATE'] == q_v['QUOTE_DATE']) ]
                        debug['size'] = len(df_filter) 
                        options_id = (str(v['SYMBOL'])+str(v['EXPIRE_DATE'])[:10]+str(q_v['QUOTE_DATE'])[:10]).replace("-",'')
                        if len(options_id) < 15:
                            print(f"""ERROR : {CSV_PATH+{d}/{f}} {v['SYMBOL']}-{v['EXPIRE_DATE']}-{q_v['QUOTE_DATE']}""",end='\r')
                            print()
                      
                        #break loops : less rows             
                        if len(df_filter) < 5 : 
                            break 
                        #add memory
                        if options_id not in [*options_qoute.keys()]:
                            options_qoute[options_id] = {}
                            options_qoute[options_id]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
                            options_qoute[options_id]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
                            options_qoute[options_id]['exp'] = df_filter['EXPIRE_DATE'].values[0]
                        
                        #rm index max : max_option_len
                        #may be initstrike out of scope
                        rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[options_id]['strike']) ].index
                        df_filter = df_filter.drop(rm_strike_index)

                        #break loops : after rm strike index           
                        if len(df_filter) < 5 : 
                            break

                        fix_values=df_filter[['QUOTE_UNIXTIME','QUOTE_READTIME','QUOTE_DATE','QUOTE_TIME_HOURS','UNDERLYING_LAST','UNDERLYING_LAST','EXPIRE_DATE','EXPIRE_UNIX','DTE','SYMBOL']].values[0]
                        #set zero 
              
                        strikeZero(df_filter,v,q_v,3 )
                        #====filed rows to max_option_len
                        z = 0
                        while len(df_filter) != max_option_len :
                            z += 1
                            if z > 50: raise "error"
                            Even= -1
                            if (len(df_filter) % 2) == 0:
                                Even = 0
                            if len(df_filter) < max_option_len :
                                zero_row = pd.DataFrame([[0]*len(df_filter.columns)], columns=df_filter.columns)
                                if Even==0 :
                                    df_filter = pd.concat([df_filter, zero_row], ignore_index=True)
                                else:
                                    df_filter = pd.concat([zero_row, df_filter], ignore_index=True)
                            elif len(df_filter) > max_option_len :
                                 df_filter = df_filter.drop(df_filter.index[Even])
            
                        #filled  0 values with values
                        df_filter[['QUOTE_UNIXTIME','QUOTE_READTIME','QUOTE_DATE','QUOTE_TIME_HOURS','UNDERLYING_LAST','UNDERLYING_LAST','EXPIRE_DATE','EXPIRE_UNIX','DTE','SYMBOL']] = fix_values
                        
                        # append
                        if len(df_filter) == max_option_len and np.sum( df_filter[OTHER_COL+SCALER_COL].isna().values ) == 0 :
                            #DATA = np.vstack((DATA ,[df_filter[OTHER_COL+SCALER_COL]]))   
                            df_filter_reindexed = df_filter.reindex(columns=OTHER_COL+SCALER_COL)
                            df_filter_reindexed['QUOTE_INDEX'] = range(1, len(df_filter_reindexed) + 1)
                            df_filter_reindexed['OPTIONS_ID'] = options_id
                            if len(DATA) :
                                DATA = pd.concat([DATA, df_filter_reindexed], ignore_index=True)
                            else:
                                DATA = df_filter_reindexed

                    # #=== clear expire options_qoute
                    for qi in list(options_qoute.keys()):
                        if options_qoute[qi]['exp'] < q_v['QUOTE_DATE']:
                            options_qoute.pop(qi)

                    #================= save ===================
                    # #==== dicrct save
                    # if os.path.exists(PARQUET_PATH):
                    #   EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                    # else:
                    #   EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    # #=== save parquet minio
                    con.register('df_table', DATA)
                    #parquet_file_path = f"s3://tensorflow/renforcement/rawdata/{v['SYMBOL']}/"
                    parquet_file_path = f"data/rawdata/{v['SYMBOL']}/"
                    os.makedirs(parquet_file_path, exist_ok=True)
                    with io.capture_output() as captured:
                        con.execute(f"COPY df_table TO '{parquet_file_path}' (FORMAT PARQUET, PARTITION_BY (EXPIRE_UNIX,QUOTE_UNIXTIME), OVERWRITE_OR_IGNORE  TRUE);")

[0/3] : qqq                           


processing:   0%|          | 0/142 [00:00<?, ?it/s]

pandas_item: 0it [00:00, ?it/s]

pandas_item: 0it [00:00, ?it/s]

pandas_item: 0it [00:00, ?it/s]

...

pandas_item: 0it [00:00, ?it/s]

pandas_item: 0it [00:00, ?it/s]

pandas_item: 0it [00:00, ?it/s]

pandas_item: 0it [00:00, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f2953faed70>>
Traceback (most recent call last):
  File "/venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [18]:
c

'C_IV'

In [22]:
EOD_CSV['C_IV'].replace('', np.nan)

0         1.950980
1         1.795070
2         1.653790
3         1.594120
4         1.527140
           ...    
16571      0.13269
16572      0.13116
16573      0.13092
16574      0.12915
16575       0.1281
Name: C_IV, Length: 16576, dtype: object

In [ ]:
EOD_CSV['C_IV'].dtype

In [ ]:
max_option_len 

In [ ]:
================= duckdb loaddata ================

In [ ]:
from minio import Minio
# Initialize MinIO client
minio_client = Minio(
    os.getenv('MINIO_HOST'),  # Change to your MinIO server address
    access_key=os.getenv('MINIO_USER'),
    secret_key=os.getenv('MINIO_SECRET'),
    secure=False  # Use False if not using HTTPS
)

In [ ]:
object_list = []
for _dir in minio_client.list_objects(BUCKET, prefix='renforcement/rawdata/'):
    for file in minio_client.list_objects(BUCKET, prefix=_dir.object_name):
        object_list.append(file.object_name)

In [ ]:
rawdata=con.execute(f"""
SELECT * FROM read_parquet('s3://tensorflow/{object_list[0]}');
""").fetchdf()